# Does MODIS have data in the Caspian?

In [2]:

import xarray as xr
f = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/MODIS/aquaDay_errorGTE03K_04km_2018-01-01.tif'
MODIS_data = xr.open_dataarray(f,engine="rasterio")
MODIS_data = MODIS_data.rename({'x':'longitude','y':'latitude'})

display(MODIS_data)

<xarray.DataArray 'band_data' (band: 1, latitude: 5400, longitude: 10800)>
[58320000 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * longitude    (longitude) float64 -180.0 -179.9 -179.9 ... 179.9 180.0 180.0
  * latitude     (latitude) float64 89.98 89.95 89.92 ... -89.92 -89.95 -89.98
    spatial_ref  int64 ...
Attributes:
    long_name:  LST_Day_1km

In [16]:
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.colors as mc
import matplotlib.colorbar as cb 


def plot_map(ds,z,vmin,vmax,cmap,logscale,title):
    
    """Scatter plot. Bit hacky. """

    try:
        x = ds.longitude
        y = ds.latitude
    except: #different sources call lat/lon different things
        x = ds.longitude_ERA
        y = ds.latitude_ERA
    
    z1 = ds[z]
    

    
    #Setup figure and axes
    fig = plt.figure(figsize=[30,20])
    gs = GridSpec(1, 2, figure=fig, width_ratios=[50,1])
    ax = fig.add_subplot(gs[0], projection=ccrs.PlateCarree(central_longitude=0))
    cax = fig.add_subplot(gs[1])
    
    #Cmaps, normalisations
    if logscale:
        norm = mc.LogNorm(vmin=vmin, vmax=vmax)
        z1 = abs(z1)
    else:
        norm = mc.Normalize(vmin=vmin, vmax=vmax)

    
    cb1 = cb.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
    
    
    
    #Plot it
    sc=ax.scatter(x,y,s=1,c=cmap(norm(z1)),
                linewidths=1, alpha=.7)
    
    
    #Tidy up
    ax.axhline(70, c='0.5', linestyle='--')
    ax.axhline(-70, c='0.5', linestyle='--') #latitude cutoffs

    ax.set_title(title)    
    ax.coastlines()

    
    ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
    ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
    ax.set_aspect('auto') #Force projection aspect ratio
    
    

    plt.show()

In [11]:
%%time
MODIS_df = MODIS_data.to_dataframe().dropna().reset_index()

CPU times: user 4.46 s, sys: 1.61 s, total: 6.07 s
Wall time: 5.47 s


In [ ]:

    
fig = plt.figure(figsize=[24,12])
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))

MODIS_data[x].plot(ax=ax, 
         cmap='jet',
         transform=ccrs.PlateCarree())
    ax.coastlines()


    plt.show()

In [21]:
ds = MODIS_data.to_dataset()

In [22]:
ds

<xarray.Dataset>
Dimensions:      (band: 1, longitude: 10800, latitude: 5400)
Coordinates:
  * band         (band) int64 1
  * longitude    (longitude) float64 -180.0 -179.9 -179.9 ... 179.9 180.0 180.0
  * latitude     (latitude) float64 89.98 89.95 89.92 ... -89.92 -89.95 -89.98
    spatial_ref  int64 0
Data variables:
    band_data    (band, latitude, longitude) float32 nan nan nan ... nan nan nan

In [ ]:
fig = plt.figure(figsize=[24,12])
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))

ds['band_data'].plot(ax=ax, 
                     cmap='jet',
                     transform=ccrs.PlateCarree())
ax.coastlines()


plt.show()

In [17]:
plot_map(MODIS_df,'band_data',vmin=200,vmax=300,cmap=plt.cm.bwr,logscale=False,title='MODIS')


KeyboardInterrupt: 